In [ ]:
import os
import cv2

import numpy as np
from PIL import Image

import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,Dropout,Dense,Flatten,MaxPooling2D

In [ ]:
# Lista todas as imagens de cada pasta
imagensUninfected=os.listdir("./data/Uninfected/")
imagensParasitized=os.listdir("./data/Parasitized/")

imagens=[]
classes=[]

for nome_imagem in imagensUninfected:
    # Lê a imagem, transformando-a em um array
    cv_imagem=cv2.imread("./data/Uninfected/"+nome_imagem)
    # Transforma o array novamente em imagem RGB
    # e a redimensiona para 50x50 pixels
    pil_imagem = Image.fromarray(cv_imagem, 'RGB')
    imagem_50px = pil_imagem.resize((50, 50))
    # Converte a imagem redimensionada em array 
    # novamente e a adiciona ao data
    imagens.append(np.array(imagem_50px))
    # Adiciona o 0 labels
    classes.append(0)

for nome_imagem in imagensParasitized:
    cv_imagem=cv2.imread("./data/Parasitized/"+nome_imagem)
    pil_imagem = Image.fromarray(cv_imagem, 'RGB')
    imagem_50px = pil_imagem.resize((50, 50))
    imagens.append(np.array(imagem_50px))
    classes.append(1)


In [ ]:
imagens=np.array(imagens)
classes=np.array(classes)

# Criamos um array aleatório com as posições
# do nosso array de imagens
posicoes=np.arange(imagens.shape[0])
np.random.shuffle(posicoes)
imagens = imagens[posicoes]
classes = classes[posicoes]

# Dividindo nossos dados em treino e teste
treino_X = (imagens[(int)(0.1*len(imagens)):]).astype('float32')/255
teste_X  = (imagens[:(int)(0.1*len(imagens))]).astype('float32')/255

# Dividindo nossas classes em treino e teste
treino_Y = keras.utils.to_categorical(classes[(int)(0.1*len(classes)):],2)
teste_Y  = keras.utils.to_categorical(classes[:(int)(0.1*len(classes))],2)



In [ ]:
# Modelo sequencial 
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",
                                                    input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2,activation="softmax"))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(treino_X,treino_Y,batch_size=50,epochs=30,verbose=1)

In [ ]:
model.evaluate(teste_X, teste_Y, verbose=1)

In [ ]:
accuracy = model.evaluate(teste_X, teste_Y, verbose=1)
print(accuracy[1])

In [ ]:
imagem = cv2.imread('./data/Parasitized/C39P4thinF_original_IMG_20150622_105253_cell_94.png')
pil_imagem = Image.fromarray(imagem, 'RGB')
imagem_50px = pil_imagem.resize((50, 50))
imagem_arr = np.array([np.array(imagem_50px)/255])
result=np.argmax(model.predict(imagem_arr))
if result == 1:
    print('Parasitized')
elif result == 0:
    print('Uninfected')
